# Federated Learning with Flower project

In [39]:
import torch as th
import os
import numpy as np
import torchvision

In [40]:
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch import optim
from collections import OrderedDict

%matplotlib inline

In [41]:
import syft as sy

In [42]:
hook= sy.TorchHook(th)

W0710 12:39:09.267355 96864 hook.py:96] Torch was already hooked... skipping hooking process


In [43]:
#create workers
w1 = sy.VirtualWorker(hook, id="w1")
w2 = sy.VirtualWorker(hook, id="w2")
secure_worker3 = sy.VirtualWorker(hook, id="secure_worker3")

In [52]:
#share pointers between workers
w1.add_workers([w2, secure_worker3])
w2.add_workers([w1, secure_worker3])
secure_worker3.add_workers([w1, w2])

W0710 12:47:42.529444 96864 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior
W0710 12:47:42.530417 96864 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0710 12:47:42.531414 96864 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0710 12:47:42.532412 96864 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0710 12:47:42.533409 96864 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0710 12:47:42.535403 96864 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker3 #tensors:0>

In [53]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 1
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and th.cuda.is_available()

th.manual_seed(args.seed)

device = th.device("cuda" if use_cuda else "cpu")

kwargs = {} if use_cuda else {}

In [60]:
data_dir = '..\\flowers\\flower_data'
# number of subprocesses to use for data loading
#num_workers = 1
# how many samples per batch to load
batch_size = 68
# percentage of training set to use as validation
valid_size = 0.2

#Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = (data_dir + 'trained_classifier.pth')
test_data = datasets.ImageFolder(data_dir + '\\valid', transform=test_transforms)

#trainloader = th.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)


federated_train_loader = sy.FederatedDataLoader(train_data.federate((w1, w2)), batch_size=16, shuffle=True, **kwargs)
testloader = th.utils.data.DataLoader(test_data, batch_size=args.test_batch_size, shuffle=False, **kwargs)



AttributeError: 'str' object has no attribute 'federate'

In [47]:
train_data.data

AttributeError: 'ImageFolder' object has no attribute 'data'